In [1]:
# import the necessary libraries (Bagging to compare with other algorithms)

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import pandasql as psql

import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 50)

import catboost
import sklearn

print("CatBoost Version     : ", catboost.__version__)
print("Scikit-Learn Version : ", sklearn.__version__)

CatBoost Version     :  0.26
Scikit-Learn Version :  0.24.1


# 01 CatBoostClassifier (Binary Classification- Breast Cancer)

In [2]:
# load the power dataset 

cancer = pd.read_csv(r"D:\iiit notes\Programming\AI\Internship practice\54 Season 06- Aug - 2021\breast_cancer.csv", header=0) 
cancer.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,TumorType
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [3]:
# Display the informationof dataset

cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [4]:
# Display the unique variables count

cancer.nunique()

mean radius                456
mean texture               479
mean perimeter             522
mean area                  539
mean smoothness            474
mean compactness           537
mean concavity             537
mean concave points        542
mean symmetry              432
mean fractal dimension     499
radius error               540
texture error              519
perimeter error            533
area error                 528
smoothness error           547
compactness error          541
concavity error            533
concave points error       507
symmetry error             498
fractal dimension error    545
worst radius               457
worst texture              511
worst perimeter            514
worst area                 544
worst smoothness           411
worst compactness          529
worst concavity            539
worst concave points       492
worst symmetry             500
worst fractal dimension    535
TumorType                    2
dtype: int64

In [5]:
# Identify the independent and Target (dependent) variables

IndepVar = []
for col in cancer.columns:
    if col != 'TumorType':
        IndepVar.append(col)

TargetVar = 'TumorType'

x = cancer[IndepVar]
y = cancer[TargetVar]

In [6]:
# Split the data into train and test (random sampling)

from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, stratify=y, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((455, 30), (114, 30), (455,), (114,))

In [7]:
# CatBoostClassifier is used for classification problems

from catboost import CatBoostClassifier
from catboost.utils import eval_metric

modelCBC = CatBoostClassifier(iterations=100, verbose=10)
modelCBC.fit(x_train, y_train, eval_set=(x_test, y_test))
modelCBC.set_feature_names(x_train)

Learning rate set to 0.071029
0:	learn: 0.5802440	test: 0.5747794	best: 0.5747794 (0)	total: 163ms	remaining: 16.1s
10:	learn: 0.1660307	test: 0.2030804	best: 0.2030804 (10)	total: 223ms	remaining: 1.81s
20:	learn: 0.0828724	test: 0.1386198	best: 0.1386198 (20)	total: 264ms	remaining: 991ms
30:	learn: 0.0526912	test: 0.1200316	best: 0.1189766 (29)	total: 303ms	remaining: 674ms
40:	learn: 0.0370860	test: 0.1182461	best: 0.1177638 (38)	total: 343ms	remaining: 494ms
50:	learn: 0.0286951	test: 0.1141475	best: 0.1141475 (50)	total: 385ms	remaining: 370ms
60:	learn: 0.0225550	test: 0.1108169	best: 0.1108169 (60)	total: 432ms	remaining: 276ms
70:	learn: 0.0177588	test: 0.1057340	best: 0.1054750 (69)	total: 473ms	remaining: 193ms
80:	learn: 0.0147118	test: 0.1048690	best: 0.1034080 (75)	total: 516ms	remaining: 121ms
90:	learn: 0.0117803	test: 0.1026150	best: 0.1026150 (90)	total: 556ms	remaining: 55ms
99:	learn: 0.0098104	test: 0.1016773	best: 0.1016773 (99)	total: 592ms	remaining: 0us

bestTe

In [8]:
# Predict the model with test data

y_pred = modelCBC.predict(x_test, prediction_type="Class")

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

Confusion matrix : 
 [[71  1]
 [ 4 38]]
Outcome values : 
 71 1 4 38
Classification report : 
               precision    recall  f1-score   support

           1       0.95      0.99      0.97        72
           0       0.97      0.90      0.94        42

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114

Accuracy : 95.6 %
Precision : 94.7 %
Recall : 98.6 %
F1 Score : 0.966
Balanced Accuracy : 94.6 %
MCC : 0.906
roc_auc_score: 0.945


In [9]:
y_pred = modelCBC.predict(x_test, prediction_type="Probability")
y_pred = pd.DataFrame(y_pred)
y_pred.head()

,0,1
0,0.998874,0.001126
1,0.000884,0.999116
2,0.991099,0.008901
3,0.798194,0.201806
4,0.998459,0.001541


# 02 CatBoostClassifier (Binary Classification- Universal Bank)

In [10]:
# Load the Universal bank data

bankdata = pd.read_csv(r"D:\iiit notes\Programming\AI\Internship practice\54 Season 06- Aug - 2021\Universalbank.csv", header=0) 
bankdata.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [11]:
# Delete the columns which are not influencing the target variable

del bankdata['ID']
del bankdata['ZIP Code']

In [12]:
# cols1 is variables - crating a dummy variables

cols1 = ['Family', 'Education']
print(cols1)

['Family', 'Education']


In [13]:
# cols2 variables - MinMaxScalar function

cols2 = ['Age', 'Experience', 'Income', 'CCAvg', 'Mortgage']
print(cols2)

['Age', 'Experience', 'Income', 'CCAvg', 'Mortgage']


In [14]:
# Create dummy variable for all range values

bankdata = pd.get_dummies(bankdata, columns=cols1)
bankdata.head().T

,0,1,2,3,4
Age,25.0,45.0,39.0,35.0,35.0
Experience,1.0,19.0,15.0,9.0,8.0
Income,49.0,34.0,11.0,100.0,45.0
CCAvg,1.6,1.5,1.0,2.7,1.0
Mortgage,0.0,0.0,0.0,0.0,0.0
Personal Loan,0.0,0.0,0.0,0.0,0.0
Securities Account,1.0,1.0,0.0,0.0,0.0
CD Account,0.0,0.0,0.0,0.0,0.0
Online,0.0,0.0,0.0,0.0,0.0
CreditCard,0.0,0.0,0.0,0.0,1.0


In [15]:
# Identify the dependent and Target variables

IndepVar = []
for col in bankdata.columns:
    if col != 'CreditCard':
        IndepVar.append(col)

TargetVar = 'CreditCard'

x = bankdata[IndepVar]
y = bankdata[TargetVar]

In [16]:
# Splitting the dataset into train and test 

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state = 42)
x_test_F1 = x_test.copy()

In [17]:
# Scaling the features by using MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

mmscaler = MinMaxScaler(feature_range=(0, 1))

x_train[cols2] = mmscaler.fit_transform(x_train[cols2])
x_train = pd.DataFrame(x_train)

x_test[cols2] = mmscaler.fit_transform(x_test[cols2])
x_test = pd.DataFrame(x_test)

In [18]:
# CatBoostClassifier is used for classification problems

from catboost import CatBoostClassifier
from catboost.utils import eval_metric

modelCBC = CatBoostClassifier(iterations=100, verbose=10)
modelCBC.fit(x_train, y_train, eval_set=(x_test, y_test))
modelCBC.set_feature_names(x_train)

Learning rate set to 0.117569
0:	learn: 0.6688767	test: 0.6689791	best: 0.6689791 (0)	total: 11.3ms	remaining: 1.12s
10:	learn: 0.5752120	test: 0.5741743	best: 0.5741743 (10)	total: 51.7ms	remaining: 418ms
20:	learn: 0.5559594	test: 0.5587001	best: 0.5587001 (20)	total: 75.1ms	remaining: 283ms
30:	learn: 0.5458331	test: 0.5522780	best: 0.5522780 (30)	total: 97.8ms	remaining: 218ms
40:	learn: 0.5382652	test: 0.5508575	best: 0.5508575 (40)	total: 122ms	remaining: 175ms
50:	learn: 0.5325087	test: 0.5509150	best: 0.5505809 (43)	total: 144ms	remaining: 139ms
60:	learn: 0.5274023	test: 0.5506267	best: 0.5504443 (59)	total: 168ms	remaining: 107ms
70:	learn: 0.5211665	test: 0.5493396	best: 0.5492185 (69)	total: 192ms	remaining: 78.3ms
80:	learn: 0.5163161	test: 0.5482259	best: 0.5482259 (80)	total: 214ms	remaining: 50.1ms
90:	learn: 0.5115173	test: 0.5480243	best: 0.5477751 (83)	total: 235ms	remaining: 23.3ms
99:	learn: 0.5084671	test: 0.5482613	best: 0.5476852 (94)	total: 254ms	remaining: 0us

In [19]:
# Predict the model with test data

y1_pred = modelCBC.predict(x_test, prediction_type="Class")

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y1_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

from sklearn.metrics import roc_curve, roc_auc_score

print('roc_auc_score:', round(roc_auc_score(y_test, y1_pred), 3))

Confusion matrix : 
 [[  69  358]
 [  11 1062]]
Outcome values : 
 69 358 11 1062
Classification report : 
               precision    recall  f1-score   support

           1       0.86      0.16      0.27       427
           0       0.75      0.99      0.85      1073

    accuracy                           0.75      1500
   macro avg       0.81      0.58      0.56      1500
weighted avg       0.78      0.75      0.69      1500

Accuracy : 75.4 %
Precision : 86.2 %
Recall : 16.2 %
F1 Score : 0.272
Balanced Accuracy : 57.6 %
MCC : 0.304
roc_auc_score: 0.576


# Random Forest Algorithm

In [20]:
# Build Random Forest classification model and Train the model using the training sets

from sklearn.ensemble import RandomForestClassifier  

modelRF = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                                 criterion='entropy', max_depth=None, max_features='auto',
                                 max_leaf_nodes=None, max_samples=None,
                                 min_impurity_decrease=0.0, min_impurity_split=None,
                                 min_samples_leaf=1, min_samples_split=2,
                                 min_weight_fraction_leaf=0.0, n_estimators=100,
                                 n_jobs=None, oob_score=False, random_state=0, verbose=0,
                                 warm_start=False)

modelRF = modelRF.fit(x_train, y_train)

# Predict the model with test data set

y2_pred = modelRF.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y2_pred 

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

print('roc_auc_score:', round(roc_auc_score(y_test, y2_pred), 3))
print('-----------------------------------------------------------------------')

Confusion matrix : 
 [[112 315]
 [109 964]]
Outcome values : 
 112 315 109 964
Classification report : 
               precision    recall  f1-score   support

           1       0.51      0.26      0.35       427
           0       0.75      0.90      0.82      1073

    accuracy                           0.72      1500
   macro avg       0.63      0.58      0.58      1500
weighted avg       0.68      0.72      0.68      1500

Accuracy : 71.7 %
Precision : 50.7 %
Recall : 26.2 %
F1 Score : 0.346
Balanced Accuracy : 58.0 %
MCC : 0.205
roc_auc_score: 0.58
-----------------------------------------------------------------------


# Logistic Regression

In [21]:
# To build the 'Logistic Regression' model with random sampling

from sklearn.linear_model import LogisticRegression

modelLR = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                             intercept_scaling=1, max_iter=100, multi_class='auto', 
                             n_jobs=None, penalty='l2', random_state=None,
                             solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

modelLR = modelLR.fit(x_train,y_train)

# Predict the model with test data set

y3_pred = modelLR.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y3_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

print('roc_auc_score:', round(roc_auc_score(y_test, y3_pred), 3))
print('-----------------------------------------------------------------------')

Confusion matrix : 
 [[  71  356]
 [  17 1056]]
Outcome values : 
 71 356 17 1056
Classification report : 
               precision    recall  f1-score   support

           1       0.81      0.17      0.28       427
           0       0.75      0.98      0.85      1073

    accuracy                           0.75      1500
   macro avg       0.78      0.58      0.56      1500
weighted avg       0.76      0.75      0.69      1500

Accuracy : 75.1 %
Precision : 80.7 %
Recall : 16.6 %
F1 Score : 0.276
Balanced Accuracy : 57.5 %
MCC : 0.289
roc_auc_score: 0.575
-----------------------------------------------------------------------


# KNN Algorithm

In [22]:
# To build the 'KNN' model 

from sklearn.neighbors import KNeighborsClassifier

modelKNN = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30,
                                p=2, metric='minkowski', metric_params=None, n_jobs=None)

modelKNN = modelKNN.fit(x_train, y_train)

# Predict the model with test data set

y4_pred = modelKNN.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y4_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

print('roc_auc_score:', round(roc_auc_score(y_test, y4_pred), 3))
print('-----------------------------------------------------------------------')

Confusion matrix : 
 [[127 300]
 [161 912]]
Outcome values : 
 127 300 161 912
Classification report : 
               precision    recall  f1-score   support

           1       0.44      0.30      0.36       427
           0       0.75      0.85      0.80      1073

    accuracy                           0.69      1500
   macro avg       0.60      0.57      0.58      1500
weighted avg       0.66      0.69      0.67      1500

Accuracy : 69.3 %
Precision : 44.1 %
Recall : 29.7 %
F1 Score : 0.355
Balanced Accuracy : 57.4 %
MCC : 0.169
roc_auc_score: 0.574
-----------------------------------------------------------------------


# SVC Algorithm - Gaussian Kernel

In [23]:
# Training the SVM algorithm 

from sklearn.svm import SVC

modelSVMGaussian = SVC(kernel='rbf', random_state = None, class_weight=None,probability=True)
modelSVMGaussian.fit(x_train, y_train)

# Predicting the values

y5_pred = modelSVMGaussian.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y5_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

print('roc_auc_score:', round(roc_auc_score(y_test, y5_pred), 3))
print('-----------------------------------------------------------------------')

Confusion matrix : 
 [[  67  360]
 [  10 1063]]
Outcome values : 
 67 360 10 1063
Classification report : 
               precision    recall  f1-score   support

           1       0.87      0.16      0.27       427
           0       0.75      0.99      0.85      1073

    accuracy                           0.75      1500
   macro avg       0.81      0.57      0.56      1500
weighted avg       0.78      0.75      0.68      1500

Accuracy : 75.3 %
Precision : 87.0 %
Recall : 15.7 %
F1 Score : 0.266
Balanced Accuracy : 57.4 %
MCC : 0.302
roc_auc_score: 0.574
-----------------------------------------------------------------------


# Decision Tree Algorithm

In [24]:
# To build the decision tree model with Over sampling 

from sklearn.tree import DecisionTreeClassifier 

modelDT = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                 max_depth=None, max_features=None, max_leaf_nodes=None,
                                 min_impurity_decrease=0.0, min_impurity_split=None,
                                 min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0,
                                 random_state=None, splitter='best')

modelDT = modelDT.fit(x_train,y_train)

# Predict with test data

y6_pred = modelDT.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values

actual = y_test

# predicted values

predicted = y6_pred

# confusion matrix

matrix = confusion_matrix(actual,predicted, labels=[1,0],sample_weight=None, normalize=None)
print('Confusion matrix : \n', matrix)

# outcome values order in sklearn

tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy

matrix = classification_report(actual,predicted,labels=[1,0])

print('Classification report : \n',matrix)

# calculating the metrics

sensitivity = round(tp/(tp+fn), 3)
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
precision = round(tp/(tp+fp), 3);
f1Score = round((2*tp/(2*tp + fp + fn)), 3);

# Matthews Correlation Coefficient (MCC). Range of values of MCC lie between -1 to +1. 
# A model with a score of +1 is a perfect model and -1 is a poor model

from math import sqrt

x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)

print('Accuracy :', round(accuracy*100, 2),'%')
print('Precision :', round(precision*100, 2),'%')
print('Recall :', round(sensitivity*100,2), '%')
print('F1 Score :', f1Score)
print('Balanced Accuracy :', round(balanced_accuracy*100, 2),'%')
print('MCC :', MCC)

# Area under ROC curve 

print('roc_auc_score:', round(roc_auc_score(y_test, y6_pred), 3))
print('-----------------------------------------------------------------------')

Confusion matrix : 
 [[180 247]
 [298 775]]
Outcome values : 
 180 247 298 775
Classification report : 
               precision    recall  f1-score   support

           1       0.38      0.42      0.40       427
           0       0.76      0.72      0.74      1073

    accuracy                           0.64      1500
   macro avg       0.57      0.57      0.57      1500
weighted avg       0.65      0.64      0.64      1500

Accuracy : 63.7 %
Precision : 37.7 %
Recall : 42.2 %
F1 Score : 0.398
Balanced Accuracy : 57.2 %
MCC : 0.139
roc_auc_score: 0.572
-----------------------------------------------------------------------


# 03 CatBoostClassifier (Multi-Class Classification- Wine)

In [25]:
# load the power dataset 

wine = pd.read_csv(r"D:\iiit notes\Programming\AI\Internship practice\54 Season 06- Aug - 2021\wine.csv", header=0) 
wine.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,WineType
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,0
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,0
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,0
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,0
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,0


In [26]:
# display the dataset information

wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    int64  
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    int64  
 13  WineT

In [27]:
# Display the number of unique values count in each variable

wine.nunique()

alcohol                         126
malic_acid                      133
ash                              79
alcalinity_of_ash                63
magnesium                        53
total_phenols                    97
flavanoids                      132
nonflavanoid_phenols             39
proanthocyanins                 101
color_intensity                 132
hue                              78
od280/od315_of_diluted_wines    122
proline                         121
WineType                          3
dtype: int64

In [28]:
# Identify the independent and Target (dependent) variables

IndepVar = []
for col in wine.columns:
    if col != 'WineType':
        IndepVar.append(col)

TargetVar = 'WineType'

x = wine[IndepVar]
y = wine[TargetVar]

In [29]:
# Split the data into train and test (random sampling)

from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, stratify=y, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((142, 13), (36, 13), (142,), (36,))

In [30]:
# Scaling the features by using MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

mmscaler = MinMaxScaler(feature_range=(0, 1))

x_train = mmscaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train)

x_test = mmscaler.fit_transform(x_test)
x_test = pd.DataFrame(x_test)

In [31]:
# CatBoostClassifier is used for multiclass classification problems

from catboost import CatBoostClassifier
from catboost.utils import eval_metric

modelCBMC = CatBoostClassifier(iterations=100, verbose=10)
modelCBMC.fit(x_train, y_train, eval_set=(x_test, y_test))
modelCBMC.set_feature_names(x_train)

Learning rate set to 0.25025
0:	learn: 0.8960927	test: 0.9271398	best: 0.9271398 (0)	total: 10.6ms	remaining: 1.05s
10:	learn: 0.2251739	test: 0.3357200	best: 0.3357200 (10)	total: 52ms	remaining: 421ms
20:	learn: 0.1054355	test: 0.2135149	best: 0.2135149 (20)	total: 76.3ms	remaining: 287ms
30:	learn: 0.0607644	test: 0.1503188	best: 0.1503188 (30)	total: 98.8ms	remaining: 220ms
40:	learn: 0.0414391	test: 0.1245208	best: 0.1245208 (40)	total: 122ms	remaining: 176ms
50:	learn: 0.0313093	test: 0.1109000	best: 0.1109000 (50)	total: 144ms	remaining: 138ms
60:	learn: 0.0243438	test: 0.0967912	best: 0.0967912 (60)	total: 169ms	remaining: 108ms
70:	learn: 0.0202625	test: 0.0896527	best: 0.0896527 (70)	total: 191ms	remaining: 78ms
80:	learn: 0.0173385	test: 0.0857718	best: 0.0857718 (80)	total: 213ms	remaining: 50ms
90:	learn: 0.0151734	test: 0.0793006	best: 0.0793006 (90)	total: 233ms	remaining: 23.1ms
99:	learn: 0.0136719	test: 0.0774022	best: 0.0774022 (99)	total: 251ms	remaining: 0us

bestT

In [32]:
# Predict model with test data 

y7_pred = modelCBMC.predict(x_test)

# confusion matrix in sklearn

from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from math import sqrt

print(confusion_matrix(y7_pred, y_test)) # Verticle is actual values & horizontal is predicted values

# Actual and predicted classes

lst_actual_class = y_test
lst_predicted_class = y7_pred

# Class = Label 0-3

lst_classes = [0, 1, 2]

# Compute multi-class confusion matrix

arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results

model_acc = [];
model_recall = [];
model_prec = [];
model_fscore = [];
model_spec = [];
model_bal_acc = [];
model_mcc = [];
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Print Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]
    
    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);
    
    x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)
    model_acc.append(accuracy);
    model_prec.append(precision);
    model_recall.append(sensitivity);
    model_fscore.append(f1Score);
    model_spec.append(specificity);
    model_bal_acc.append(balanced_accuracy);
    model_mcc.append(MCC);
    
    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Accuracy: {0}".format(accuracy));    # Accuracy score
    print("Precision: {0}".format(precision)); # Precision score
    print("Sensitivity: {0}".format(sensitivity)); # Recall score
    print("F1-Score: {0}".format(f1Score)); # F1 score
    print("Specificity: {0}".format(specificity)); # True Nagative Rate
    print("Balanced Accuracy: {0}".format(balanced_accuracy)); # Balance accuracy score
    print("MCC: {0}\n".format(MCC)); # Matthews Correlation Coefficient

[[11  0  0]
 [ 1 14  0]
 [ 0  0 10]]
Print Class: 0
TP=11, FP=0, TN=24, FN=1
Accuracy: 0.972
Precision: 1.0
Sensitivity: 0.917
F1-Score: 0.957
Specificity: 1.0
Balanced Accuracy: 0.958
MCC: 0.938

Print Class: 1
TP=14, FP=1, TN=21, FN=0
Accuracy: 0.972
Precision: 0.933
Sensitivity: 1.0
F1-Score: 0.966
Specificity: 0.955
Balanced Accuracy: 0.978
MCC: 0.944

Print Class: 2
TP=10, FP=0, TN=26, FN=0
Accuracy: 1.0
Precision: 1.0
Sensitivity: 1.0
F1-Score: 1.0
Specificity: 1.0
Balanced Accuracy: 1.0
MCC: 1.0



In [33]:
# OVERALL - FINAL PREDICTION PERFORMANCE

# importing mean() 

from statistics import mean
import math

print("Overall Performance Prediction:");
print("Accuracy: {0}%".format(round(mean(model_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(model_prec)*100, 4)));
print("Recall or Sensitivity: {0}%".format(round(mean(model_recall)*100, 4)));
print("F1-Score: {0}".format(round(mean(model_fscore), 4)));
print("Specificity or True Nagative Rate: {0}%".format(round(mean(model_spec)*100, 4)));
print("Balanced Accuracy: {0}%\n".format(round(mean(model_bal_acc)*100, 4)));
print("MCC: {0}\n".format(round(mean(model_mcc), 4)));

Overall Performance Prediction:
Accuracy: 98.1333%
Precision: 97.7667%
Recall or Sensitivity: 97.2333%
F1-Score: 0.9743
Specificity or True Nagative Rate: 98.5%
Balanced Accuracy: 97.8667%

MCC: 0.9607



# Random Forest Algorithm

In [34]:
from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier(criterion='gini', n_estimators=100, random_state=0)

modelRF.fit(x_train,y_train)

# Predict model with test data 

y8_pred =modelRF.predict(x_test)
y8_pred_proba = modelRF.predict_proba(x_test)

# confusion matrix in sklearn

from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from math import sqrt

print(confusion_matrix(y8_pred, y_test)) # Verticle is actual values & horizontal is predicted values

# Actual and predicted classes

lst_actual_class = y_test
lst_predicted_class = y8_pred

# Class = Label 0-3

lst_classes = [0, 1, 2]

# Compute multi-class confusion matrix

arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results

model_acc = [];
model_recall = [];
model_prec = [];
model_fscore = [];
model_spec = [];
model_bal_acc = [];
model_mcc = [];
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Print Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]
    
    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);
    
    precision = round(tp/(tp+fp), 3);
    f1Score = round((2*tp/(2*tp + fp + fn)), 3);
    
    x = (tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)
    MCC = round(((tp * tn) - (fp * fn)) / sqrt(x), 3)
    model_acc.append(accuracy);
    model_prec.append(precision);
    model_recall.append(sensitivity);
    model_fscore.append(f1Score);
    model_spec.append(specificity);
    model_bal_acc.append(balanced_accuracy);
    model_mcc.append(MCC);
    
    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Accuracy: {0}".format(accuracy));    # Accuracy score
    print("Precision: {0}".format(precision)); # Precision score
    print("Sensitivity: {0}".format(sensitivity)); # Recall score
    print("F1-Score: {0}".format(f1Score)); # F1 score
    print("Specificity: {0}".format(specificity)); # True Nagative Rate
    print("Balanced Accuracy: {0}".format(balanced_accuracy)); # Balance accuracy score
    print("MCC: {0}\n".format(MCC)); # Matthews Correlation Coefficient

[[11  0  0]
 [ 1 14  0]
 [ 0  0 10]]
Print Class: 0
TP=11, FP=0, TN=24, FN=1
Accuracy: 0.972
Precision: 1.0
Sensitivity: 0.917
F1-Score: 0.957
Specificity: 1.0
Balanced Accuracy: 0.958
MCC: 0.938

Print Class: 1
TP=14, FP=1, TN=21, FN=0
Accuracy: 0.972
Precision: 0.933
Sensitivity: 1.0
F1-Score: 0.966
Specificity: 0.955
Balanced Accuracy: 0.978
MCC: 0.944

Print Class: 2
TP=10, FP=0, TN=26, FN=0
Accuracy: 1.0
Precision: 1.0
Sensitivity: 1.0
F1-Score: 1.0
Specificity: 1.0
Balanced Accuracy: 1.0
MCC: 1.0



In [35]:
# OVERALL - FINAL PREDICTION PERFORMANCE

# importing mean() 

from statistics import mean
import math

print("Overall Performance Prediction:");
print("Accuracy: {0}%".format(round(mean(model_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(model_prec)*100, 4)));
print("Recall or Sensitivity: {0}%".format(round(mean(model_recall)*100, 4)));
print("F1-Score: {0}".format(round(mean(model_fscore), 4)));
print("Specificity or True Nagative Rate: {0}%".format(round(mean(model_spec)*100, 4)));
print("Balanced Accuracy: {0}%\n".format(round(mean(model_bal_acc)*100, 4)));
print("MCC: {0}\n".format(round(mean(model_mcc), 4)));

Overall Performance Prediction:
Accuracy: 98.1333%
Precision: 97.7667%
Recall or Sensitivity: 97.2333%
F1-Score: 0.9743
Specificity or True Nagative Rate: 98.5%
Balanced Accuracy: 97.8667%

MCC: 0.9607

